# 🛢️ Well Production Simulation: Using Real Historical Data

This notebook demonstrates how to simulate wells using real data from:
- **Dimension Table** (`qi_di_b_factor_dimension.csv`): Well parameters (Qi, Di, b_factor, start_year)
- **Fact Table** (`houston_fact_combined_cleaned.csv`): Actual daily production records

## Workflow:
1. Get **Qi, Di, b_factor** from dimension table
2. Get **latest_production_date** from fact table
3. Calculate **days_on_production** = `latest_production_date - start_year`
4. Use actual production data where available
5. Simulate remaining forecast period using decline curve

```
|-------- Historical Data --------|-------- Forecast --------|
   start_year              latest_prod_date          sim_end_date
       ↓                         ↓                         ↓
   [Actual values from fact]  [Switch to decline curve simulation]
```

---
## 1. Setup & Imports

In [ ]:
# Standard imports
import sys
import math
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional

# Data analysis
import polars as pl
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Set up paths
SIMULATION_ROOT = Path(r"D:\Mu\Simulation")
DATA_DIR = SIMULATION_ROOT / "data"
sys.path.insert(0, str(SIMULATION_ROOT))

# Configure display
pl.Config.set_tbl_rows(20)
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:,.2f}'.format)

# Matplotlib style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

print("✅ Standard imports loaded")

In [ ]:
# Import simulation modules
from simulation import SimulationRunner, BasinModel, WellAgent
from config.simulation_config import (
    WellType, DeclineType, DriveMechanism, WellConfiguration,
    WELL_TYPE_PARAMETERS, CONSTANTS,
)
print("✅ Simulation modules loaded successfully!")

---
## 2. Load Data

In [ ]:
# Load dimension table (has Qi, Di, b_factor, start_year)
print("📊 Loading dimension data...")
dim = pl.read_csv(DATA_DIR / "qi_di_b_factor_dimension.csv")
print(f"✅ Dimension table loaded: {dim.shape[0]:,} rows x {dim.shape[1]} columns")

print("\n📋 Dimension Table Columns:")
for col in dim.columns:
    print(f"   - {col}")

In [ ]:
# Load fact table (has actual daily production)
print("📊 Loading fact data (this may take a moment for large file)...")
fact = pl.read_csv(DATA_DIR / "houston_fact_combined_cleaned.csv")
print(f"✅ Fact table loaded: {fact.shape[0]:,} rows x {fact.shape[1]} columns")

print("\n📋 Fact Table Columns:")
for col in fact.columns:
    print(f"   - {col}")

In [ ]:
# Explore dimension table structure
print("📊 DIMENSION TABLE - Sample Data:")
print("=" * 80)
dim.head(5)

In [ ]:
# Explore fact table structure
print("📊 FACT TABLE - Sample Data:")
print("=" * 80)
fact.head(5)

---
## 3. Get Well Summary from Fact Table

For each well (API), find:
- First production date
- **Latest production date** (when actual data ends)
- Total days of data

In [ ]:
# Convert date strings to dates and get per-well summary
fact_with_dates = fact.with_columns([
    pl.col('production_vol_date').str.to_date().alias('prod_date')
])

# Get summary for each well
well_summary = (
    fact_with_dates
    .group_by('api')
    .agg([
        pl.col('prod_date').min().alias('first_prod_date'),
        pl.col('prod_date').max().alias('latest_prod_date'),  # KEY: When actual data ends
        pl.col('bopd').first().alias('first_bopd'),
        pl.col('bopd').last().alias('last_bopd'),  # Last known production
        pl.col('bopd').mean().alias('avg_bopd'),
        pl.col('bopd').max().alias('max_bopd'),
        pl.col('bopd').sum().alias('total_oil'),
        pl.col('play_type').first().alias('play_type'),
        pl.col('basin').first().alias('basin'),
        pl.col('production_status').last().alias('status'),
        pl.len().alias('days_of_data'),  # Number of days with actual data
    ])
    .sort('total_oil', descending=True)
)

print(f"📊 Total unique wells: {well_summary.shape[0]:,}")
print("\n🔝 Top 10 wells by total production:")
well_summary.head(10)

---
## 4. Merge with Dimension Table

Get Qi, Di, b_factor, and start_year from dimension table.

In [ ]:
# Check what columns are in dimension table that we need
print("📋 Looking for key columns in dimension table...")
print(f"\nAll columns: {dim.columns}")

# Find the API/well identifier column
api_cols = [c for c in dim.columns if 'api' in c.lower() or 'well' in c.lower() or 'id' in c.lower()]
print(f"\nPossible API columns: {api_cols}")

# Find Qi/Di/b_factor columns
qi_cols = [c for c in dim.columns if 'qi' in c.lower() or 'initial' in c.lower()]
di_cols = [c for c in dim.columns if 'di' in c.lower() or 'decline' in c.lower()]
b_cols = [c for c in dim.columns if 'b_' in c.lower() or 'bfactor' in c.lower()]
year_cols = [c for c in dim.columns if 'year' in c.lower() or 'start' in c.lower() or 'date' in c.lower()]

print(f"\nQi columns: {qi_cols}")
print(f"Di columns: {di_cols}")
print(f"b_factor columns: {b_cols}")
print(f"Year/Date columns: {year_cols}")

In [ ]:
# Show sample of dimension data to identify the exact column names
print("📊 Dimension table sample with all columns:")
dim.head(3)

In [ ]:
# Define column mappings based on what we found
# (Update these based on actual column names in your dimension table)

# Try to identify columns - will adjust based on actual data
DIM_API_COL = 'api' if 'api' in dim.columns else dim.columns[0]  # Well identifier
DIM_QI_COL = 'Qi' if 'Qi' in dim.columns else 'qi'  # Initial production
DIM_DI_COL = 'Di' if 'Di' in dim.columns else 'di'  # Decline rate
DIM_B_COL = 'b_factor' if 'b_factor' in dim.columns else 'b'  # b-factor
DIM_START_YEAR_COL = 'start_year' if 'start_year' in dim.columns else None

print(f"Using columns:")
print(f"  API: {DIM_API_COL}")
print(f"  Qi:  {DIM_QI_COL}")
print(f"  Di:  {DIM_DI_COL}")
print(f"  b:   {DIM_B_COL}")
print(f"  Start Year: {DIM_START_YEAR_COL}")

In [ ]:
# Select the columns we need from dimension table
dim_subset = dim.select([
    pl.col(DIM_API_COL).alias('api'),
    pl.col(DIM_QI_COL).alias('qi'),
    pl.col(DIM_DI_COL).alias('di'),
    pl.col(DIM_B_COL).alias('b_factor'),
])

# If start_year column exists, include it
if DIM_START_YEAR_COL and DIM_START_YEAR_COL in dim.columns:
    dim_subset = dim.select([
        pl.col(DIM_API_COL).alias('api'),
        pl.col(DIM_QI_COL).alias('qi'),
        pl.col(DIM_DI_COL).alias('di'),
        pl.col(DIM_B_COL).alias('b_factor'),
        pl.col(DIM_START_YEAR_COL).alias('start_year'),
    ])

print("📊 Dimension subset:")
dim_subset.head(5)

In [ ]:
# Merge well summary with dimension data
# This gives us: actual data dates + decline curve parameters

wells_merged = well_summary.join(
    dim_subset,
    on='api',
    how='left'
)

print(f"📊 Merged wells: {wells_merged.shape[0]:,}")
print(f"   With Qi data: {wells_merged.filter(pl.col('qi').is_not_null()).shape[0]:,}")

# Filter to wells that have both fact and dimension data
wells_with_params = wells_merged.filter(
    pl.col('qi').is_not_null() & 
    pl.col('di').is_not_null() &
    (pl.col('qi') > 0)
)

print(f"   Wells with complete params: {wells_with_params.shape[0]:,}")
wells_with_params.head(10)

---
## 5. Calculate Days on Production

```
days_on_production = latest_prod_date - start_year
```

If `start_year` is not available, we'll use `first_prod_date` from the fact table.

In [ ]:
# Calculate days on production
# Use start_year if available, otherwise use first_prod_date from fact table

if 'start_year' in wells_with_params.columns:
    # Convert start_year to date (assume Jan 1 of that year)
    wells_with_days = wells_with_params.with_columns([
        # If start_year is a year number, create date from it
        pl.when(pl.col('start_year').is_not_null())
        .then(
            pl.date(pl.col('start_year').cast(pl.Int32), 1, 1)
        )
        .otherwise(pl.col('first_prod_date'))
        .alias('production_start')
    ])
else:
    # Use first_prod_date from fact table as start
    wells_with_days = wells_with_params.with_columns([
        pl.col('first_prod_date').alias('production_start')
    ])

# Calculate days on production
wells_with_days = wells_with_days.with_columns([
    (pl.col('latest_prod_date') - pl.col('production_start')).dt.total_days().alias('days_on_production')
])

print("📊 Wells with days_on_production calculated:")
wells_with_days.select([
    'api', 'production_start', 'latest_prod_date', 'days_on_production', 
    'qi', 'di', 'b_factor', 'last_bopd'
]).head(10)

---
## 6. Select a Well for Testing

In [ ]:
# Select a well with good data for demonstration
# Choose one with reasonable Qi, days of data, and active status

good_wells = wells_with_days.filter(
    (pl.col('qi') > 50) & 
    (pl.col('qi') < 2000) &
    (pl.col('days_on_production') > 180) &  # At least 6 months of history
    (pl.col('di') > 0) &
    (pl.col('di') < 2)  # Reasonable decline rate
).sort('total_oil', descending=True)

print(f"📊 Wells meeting criteria: {good_wells.shape[0]}")
print("\n🔝 Top 5 candidates:")
good_wells.select([
    'api', 'play_type', 'qi', 'di', 'b_factor', 
    'days_on_production', 'last_bopd', 'total_oil'
]).head(5)

In [ ]:
# Select the first good well for our test
test_well_row = good_wells.row(0, named=True)

TEST_API = test_well_row['api']
TEST_QI = test_well_row['qi']  # From dimension table
TEST_DI = test_well_row['di']  # From dimension table  
TEST_B = test_well_row['b_factor'] if test_well_row['b_factor'] else 0.5
TEST_PROD_START = test_well_row['production_start']
TEST_LATEST_PROD = test_well_row['latest_prod_date']  # When actual data ends
TEST_DAYS_ON_PROD = test_well_row['days_on_production']
TEST_LAST_BOPD = test_well_row['last_bopd']  # Last observed production

print(f"📍 Selected Well for Testing:")
print("=" * 60)
print(f"   API: {TEST_API}")
print(f"   Play Type: {test_well_row['play_type']}")
print(f"\n📊 From DIMENSION table:")
print(f"   Qi (Initial Production): {TEST_QI:.1f} bbl/day")
print(f"   Di (Decline Rate): {TEST_DI:.4f}")
print(f"   b_factor: {TEST_B:.2f}")
print(f"\n📅 From FACT table:")
print(f"   Production Start: {TEST_PROD_START}")
print(f"   Latest Production Date: {TEST_LATEST_PROD} ← Actual data ends here")
print(f"   Days on Production: {TEST_DAYS_ON_PROD:,}")
print(f"   Last Observed Rate: {TEST_LAST_BOPD:.1f} bbl/day")

---
## 7. Get Actual Production History for This Well

In [ ]:
# Get all actual production records for this well
well_fact_data = fact_with_dates.filter(pl.col('api') == TEST_API).sort('prod_date')

print(f"📊 Actual production records for API {TEST_API}: {well_fact_data.shape[0]} days")
print(f"\nFirst 5 records:")
display(well_fact_data.head(5))
print(f"\nLast 5 records:")
display(well_fact_data.tail(5))

In [ ]:
# Convert to dictionary for simulation (date string -> production value)
actual_production_dict = {}
for row in well_fact_data.iter_rows(named=True):
    date_str = row['prod_date'].strftime('%Y-%m-%d')
    bopd = row['bopd']
    if bopd is not None:
        actual_production_dict[date_str] = float(bopd)

print(f"✅ Created production dictionary with {len(actual_production_dict)} entries")
print(f"   Date range: {min(actual_production_dict.keys())} to {max(actual_production_dict.keys())}")

In [ ]:
# Visualize actual production history
dates = [datetime.strptime(d, '%Y-%m-%d') for d in sorted(actual_production_dict.keys())]
values = [actual_production_dict[d.strftime('%Y-%m-%d')] for d in dates]

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(dates, values, 'b-', linewidth=1, alpha=0.8, label='Actual Production')
ax.fill_between(dates, values, alpha=0.2, color='blue')

# Mark the end of actual data
ax.axvline(x=dates[-1], color='red', linestyle='--', linewidth=2, 
           label=f'Actual Data Ends ({dates[-1].strftime("%Y-%m-%d")})')

# Add Qi reference line
ax.axhline(y=TEST_QI, color='green', linestyle=':', linewidth=1.5, 
           label=f'Qi from dim table = {TEST_QI:.0f}')

ax.set_xlabel('Date')
ax.set_ylabel('Production (bbl/day)')
ax.set_title(f'Actual Production History - API {TEST_API}', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 8. Create Simulation with Historical Data + Forecast

Now we create a simulation that:
1. Uses **actual data** while available
2. Switches to **decline curve** after `latest_prod_date`

In [ ]:
# Define simulation period
# Start from when we have actual data, continue 1 year past the last actual data

# Convert to datetime objects
if isinstance(TEST_PROD_START, str):
    sim_start = datetime.strptime(TEST_PROD_START, '%Y-%m-%d')
else:
    sim_start = datetime(TEST_PROD_START.year, TEST_PROD_START.month, TEST_PROD_START.day)

if isinstance(TEST_LATEST_PROD, str):
    last_data_date = datetime.strptime(TEST_LATEST_PROD, '%Y-%m-%d')
else:
    last_data_date = datetime(TEST_LATEST_PROD.year, TEST_LATEST_PROD.month, TEST_LATEST_PROD.day)

# End simulation 1 year after last actual data
sim_end = last_data_date + timedelta(days=365)

print(f"📅 Simulation Configuration:")
print("=" * 60)
print(f"   Start Date: {sim_start.date()}")
print(f"   Last Actual Data: {last_data_date.date()} ← Historical ends here")
print(f"   End Date: {sim_end.date()} ← 1 year of forecast")
print(f"\n   Days with actual data: {(last_data_date - sim_start).days}")
print(f"   Days to forecast: {(sim_end - last_data_date).days}")

In [ ]:
# Map play_type to well_type
def map_play_type_to_well_type(play_type: str) -> str:
    mapping = {
        'CONVENTIONAL ONSHORE': 'Conventional Onshore',
        'CONVENTIONAL OFFSHORE': 'Conventional Offshore',
        'SHALE': 'Shale Oil',
        'SHALE OIL': 'Shale Oil',
        'TIGHT': 'Tight Oil',
        'TIGHT OIL': 'Tight Oil',
    }
    return mapping.get(str(play_type).upper(), 'Conventional Onshore')

well_type = map_play_type_to_well_type(test_well_row['play_type'])
print(f"Well type: {test_well_row['play_type']} → {well_type}")

In [ ]:
# Create simulation runner
runner = SimulationRunner(
    total_recoverable_resources=100_000_000,
    start_date=sim_start,
    end_date=sim_end,
    basin_name='Houston Embayment',
    market_center='Houston',
    country='US',
)

print(f"✅ SimulationRunner created")
print(f"   Total steps: {runner.total_steps}")

In [ ]:
# Add well with historical data
# KEY PARAMETERS:
# - actual_production_data: Dict of actual daily production
# - last_known_qi: Production to use when switching to simulation
# - last_known_di: Decline rate to use when switching to simulation
# - last_data_date: When actual data ends

well_state = runner.add_well(
    # Identification
    api_number=TEST_API,
    
    # Well type and configuration
    well_type=well_type,
    configuration=WellConfiguration.HORIZONTAL.value,
    decline_type=DeclineType.HYPERBOLIC.value,
    
    # === Production parameters from DIMENSION TABLE ===
    initial_production=TEST_QI,  # Qi from dim table
    b_factor=TEST_B,             # b_factor from dim table
    nominal_decline_rate=TEST_DI, # Di from dim table
    
    # Dates
    production_start_date=sim_start,
    
    # Location
    location='Houston Embayment',
    basin='Gulf Coast',
    
    # === HISTORICAL PRODUCTION DATA ===
    actual_production_data=actual_production_dict,
    
    # === FALLBACK PARAMETERS (for when historical data ends) ===
    last_known_qi=TEST_LAST_BOPD,    # Use last observed production as starting Qi
    last_known_di=TEST_DI / 12,      # Convert annual to monthly decline
    last_data_date=last_data_date,   # When actual data ends
    
    # Mark as existing well
    is_new=False,
)

print(f"\n✅ Well added to simulation:")
print(f"   ID: {well_state['id']}")
print(f"   Type: {well_state['well_type']}")
print(f"   Qi: {TEST_QI:.1f} bbl/day")
print(f"   Di: {TEST_DI:.4f} (annual)")
print(f"   b_factor: {TEST_B:.2f}")
print(f"   has_historical_data: True")

In [ ]:
# Run simulation
print("🚀 Running simulation...\n")

results = []
runner.start()

step = 0
while runner.model.simulation_active:
    state = runner.step()
    step += 1
    
    sim_state = state['state']
    current_date = datetime.fromisoformat(sim_state['current_date'])
    well_data = sim_state['basin']['wells'][0]
    
    # Check if using actual data or simulation
    date_str = current_date.strftime('%Y-%m-%d')
    using_actual = date_str in actual_production_dict
    
    results.append({
        'date': current_date,
        'daily_production': well_data['daily_production'],
        'potential_production': well_data['daily_potential_production'],
        'cumulative': well_data['cumulative_production'],
        'source': 'Actual' if using_actual else 'Simulated',
    })
    
    # Progress updates at key points
    if step % 100 == 0 or step == 1:
        source_label = "ACTUAL" if using_actual else "SIMULATED"
        print(f"  Day {step:4d} ({date_str}): {well_data['daily_production']:>6.1f} bbl/day [{source_label}]")

print(f"\n✅ Simulation complete! {step} days.")

df_results = pd.DataFrame(results)
df_results.set_index('date', inplace=True)

---
## 9. Visualize Results: Historical → Forecast Transition

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Split data by source
actual_mask = df_results['source'] == 'Actual'
simulated_mask = df_results['source'] == 'Simulated'

# 1. Daily Production - Actual vs Simulated
ax1 = axes[0, 0]
ax1.plot(df_results[actual_mask].index, df_results[actual_mask]['daily_production'], 
         'b-', linewidth=1.5, label='Actual Data', alpha=0.8)
ax1.plot(df_results[simulated_mask].index, df_results[simulated_mask]['daily_production'], 
         'r-', linewidth=2, label='Simulated (Decline Curve)', alpha=0.8)

ax1.axvline(x=last_data_date, color='gray', linestyle='--', linewidth=2, 
            label=f'Data Ends ({last_data_date.strftime("%Y-%m-%d")})')
ax1.axhline(y=TEST_QI, color='green', linestyle=':', alpha=0.5, label=f'Qi = {TEST_QI:.0f}')

ax1.set_xlabel('Date')
ax1.set_ylabel('Production (bbl/day)')
ax1.set_title(f'API {TEST_API}: Actual Data → Simulation Transition', fontsize=14, fontweight='bold')
ax1.legend(loc='upper right')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax1.grid(True, alpha=0.3)

# 2. Cumulative Production
ax2 = axes[0, 1]
ax2.fill_between(df_results.index, df_results['cumulative'] / 1000, alpha=0.3, color='green')
ax2.plot(df_results.index, df_results['cumulative'] / 1000, 'g-', linewidth=2)
ax2.axvline(x=last_data_date, color='gray', linestyle='--', linewidth=2)
ax2.set_xlabel('Date')
ax2.set_ylabel('Cumulative Production (thousand bbl)')
ax2.set_title('Cumulative Production', fontsize=14, fontweight='bold')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax2.grid(True, alpha=0.3)

# 3. Source breakdown (pie chart)
ax3 = axes[1, 0]
source_counts = df_results['source'].value_counts()
colors = ['#3498db', '#e74c3c']
ax3.pie(source_counts.values, labels=source_counts.index, autopct='%1.1f%%', 
        colors=colors, explode=[0.05, 0], shadow=True, startangle=90)
ax3.set_title('Data Source Breakdown', fontsize=14, fontweight='bold')

# 4. Zoom on transition point
ax4 = axes[1, 1]
# Show 30 days before and after transition
transition_start = last_data_date - timedelta(days=30)
transition_end = last_data_date + timedelta(days=60)
transition_data = df_results.loc[transition_start:transition_end]

t_actual = transition_data[transition_data['source'] == 'Actual']
t_simulated = transition_data[transition_data['source'] == 'Simulated']

ax4.plot(t_actual.index, t_actual['daily_production'], 'b-', linewidth=2, label='Actual')
ax4.plot(t_simulated.index, t_simulated['daily_production'], 'r-', linewidth=2, label='Simulated')
ax4.axvline(x=last_data_date, color='gray', linestyle='--', linewidth=2, label='Transition Point')
ax4.scatter([last_data_date], [TEST_LAST_BOPD], color='orange', s=100, zorder=5, 
            label=f'Last Actual = {TEST_LAST_BOPD:.0f}')

ax4.set_xlabel('Date')
ax4.set_ylabel('Production (bbl/day)')
ax4.set_title('Transition Point Detail', fontsize=14, fontweight='bold')
ax4.legend()
ax4.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.setp(ax4.xaxis.get_majorticklabels(), rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\n" + "=" * 70)
print("📊 SIMULATION SUMMARY")
print("=" * 70)
print(f"\nData Sources:")
print(f"   Days with ACTUAL data: {actual_mask.sum()}")
print(f"   Days with SIMULATED data: {simulated_mask.sum()}")
print(f"\nProduction Totals:")
print(f"   Total Cumulative: {df_results['cumulative'].iloc[-1]:,.0f} bbl")
print(f"   From Actual Data: {df_results[actual_mask]['daily_production'].sum():,.0f} bbl")
print(f"   From Simulation: {df_results[simulated_mask]['daily_production'].sum():,.0f} bbl")
print(f"\nTransition:")
print(f"   Last actual production: {TEST_LAST_BOPD:.1f} bbl/day")
if simulated_mask.sum() > 0:
    print(f"   First simulated production: {df_results[simulated_mask]['daily_production'].iloc[0]:.1f} bbl/day")
    print(f"   Final simulated production: {df_results[simulated_mask]['daily_production'].iloc[-1]:.1f} bbl/day")

---
## 10. Summary: The Complete Workflow

```
┌─────────────────────────────────────────────────────────────────────┐
│                         DATA PREPARATION                            │
├─────────────────────────────────────────────────────────────────────┤
│ 1. DIMENSION TABLE (qi_di_b_factor_dimension.csv)                   │
│    → Get: Qi, Di, b_factor, start_year                             │
│                                                                     │
│ 2. FACT TABLE (houston_fact_combined_cleaned.csv)                   │
│    → Get: latest_production_date, actual daily production           │
│                                                                     │
│ 3. CALCULATE:                                                       │
│    days_on_production = latest_prod_date - start_year               │
│    last_bopd = last observed production value                       │
└─────────────────────────────────────────────────────────────────────┘
                                ↓
┌─────────────────────────────────────────────────────────────────────┐
│                         SIMULATION CONFIG                           │
├─────────────────────────────────────────────────────────────────────┤
│ runner.add_well(                                                    │
│     # From dimension table                                          │
│     initial_production=Qi,                                          │
│     nominal_decline_rate=Di,                                        │
│     b_factor=b_factor,                                              │
│                                                                     │
│     # From fact table - actual daily production                     │
│     actual_production_data={'2024-01-01': 500, ...},               │
│                                                                     │
│     # Fallback parameters for when data ends                        │
│     last_known_qi=last_bopd,  # Use last actual value              │
│     last_known_di=Di/12,       # Monthly decline                    │
│     last_data_date=latest_production_date,                          │
│                                                                     │
│     is_new=False,  # Existing well with history                     │
│ )                                                                   │
└─────────────────────────────────────────────────────────────────────┘
                                ↓
┌─────────────────────────────────────────────────────────────────────┐
│                         SIMULATION RUN                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  |-------- Historical ---------|----- Forecast -----|               │
│  start_year           latest_prod_date        sim_end               │
│       ↓                      ↓                    ↓                 │
│  [Use actual values]  [Switch to decline curve simulation]          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Save results
output_dir = SIMULATION_ROOT / "output"
output_dir.mkdir(exist_ok=True)

df_results.to_csv(output_dir / f"well_{TEST_API}_simulation.csv")
print(f"💾 Results saved to: {output_dir / f'well_{TEST_API}_simulation.csv'}")